In [1]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
import torch.nn.functional as F
warnings.filterwarnings("ignore")

In [2]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=4)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)
parser_train.add_argument('--k', type=int, default=2)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)


parser_train.add_argument('--warm_up_epochs', type=int, default=15)
parser_train.add_argument('--selection_epochs', type=int, default=4)
parser_train.add_argument('--importance', type=float, default=0)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:1' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model2/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
parser_train.add_argument('--decay', type=float, default=0.75)
args_train = parser_train.parse_args(args=[])

In [3]:
from Server import  Server_mul,Server
from Clients import Client_mul,Client
from Train import Trainer,EWCLoss,evaluation,PinballLoss
from utils import get_data, plot_prob_result,seed_everything

In [4]:
seed_everything(0)

In [5]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client_mul(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:06<00:00,  1.07it/s]


In [6]:
server = Server_mul(args_train,clients)

In [7]:
server.fed_train() 

test performance: [0.14660560411133178, 0.1470871785374945, 0.17529371837536767, 0.16002381862179466, 0.15218704218070392, 0.16332173204585299, 0.1624772938555234]
False [False, False]
ewc: False
wf1 model: 0
Epoch: 0 | Loss: 0.0630
wf1 model: 1
Epoch: 0 | Loss: 0.0592
ewc: False
wf2 model: 0
Epoch: 0 | Loss: 0.0626
wf2 model: 1
Epoch: 0 | Loss: 0.0797
ewc: False
wf3 model: 0
Epoch: 0 | Loss: 0.0845
wf3 model: 1
Epoch: 0 | Loss: 0.0870
ewc: False
wf4 model: 0
Epoch: 0 | Loss: 0.0872
wf4 model: 1
Epoch: 0 | Loss: 0.0759
ewc: False
wf5 model: 0
Epoch: 0 | Loss: 0.0752
wf5 model: 1
Epoch: 0 | Loss: 0.0858
ewc: False
wf6 model: 0
Epoch: 0 | Loss: 0.0797
wf6 model: 1
Epoch: 0 | Loss: 0.0776
ewc: False
wf7 model: 0
Epoch: 0 | Loss: 0.0775
wf7 model: 1
Epoch: 0 | Loss: 0.0885
Client 0 test loss: [0.07275691115350094, 0.07253277969529044]
Client 1 test loss: [0.07832480714006244, 0.07824687413730712]
Client 2 test loss: [0.09598511401212441, 0.09554408771811791]
Client 3 test loss: [0.08376877

[ANN(
   (hidden_layers): ModuleList(
     (0): Linear(in_features=293, out_features=64, bias=True)
     (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
   )
   (relu): ReLU()
   (output): Linear(in_features=64, out_features=9, bias=True)
 ),
 ANN(
   (hidden_layers): ModuleList(
     (0): Linear(in_features=293, out_features=64, bias=True)
     (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
   )
   (relu): ReLU()
   (output): Linear(in_features=64, out_features=9, bias=True)
 )]

In [8]:
#0.5
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss[server.index_set[i]])
    fed_local_preds.append(fed_local_pred[server.index_set[i]])
    fed_local_models.append(fed_local_model[server.index_set[i]])
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.04661360359431742, 0.05353561454541879, 0.0576981087279034, 0.05322195598472879, 0.052451892526284474, 0.05084744619511782, 0.05232491294776842]
0.05238479064593416


In [13]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune(server.index_set[i],ewc_flag=False,importance=0.1,fine_tune_epochs=20)
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/20] Train Loss: 0.0423 Val Loss: 0.0433
Epoch [2/20] Train Loss: 0.0419 Val Loss: 0.0430
Epoch [3/20] Train Loss: 0.0417 Val Loss: 0.0429
Epoch [4/20] Train Loss: 0.0415 Val Loss: 0.0428
Epoch [5/20] Train Loss: 0.0415 Val Loss: 0.0427
Epoch [6/20] Train Loss: 0.0413 Val Loss: 0.0427
Epoch [7/20] Train Loss: 0.0413 Val Loss: 0.0426
Epoch [8/20] Train Loss: 0.0413 Val Loss: 0.0429
Epoch [9/20] Train Loss: 0.0411 Val Loss: 0.0426
Epoch [10/20] Train Loss: 0.0410 Val Loss: 0.0427
Epoch [11/20] Train Loss: 0.0410 Val Loss: 0.0424
Epoch [12/20] Train Loss: 0.0409 Val Loss: 0.0424
Epoch [13/20] Train Loss: 0.0408 Val Loss: 0.0424
Epoch [14/20] Train Loss: 0.0408 Val Loss: 0.0423
Epoch [15/20] Train Loss: 0.0407 Val Loss: 0.0424
Epoch [16/20] Train Loss: 0.0407 Val Loss: 0.0423
Epoch [17/20] Train Loss: 0.0406 Val Loss: 0.0423
Epoch [18/20] Train Loss: 0.0405 Val Loss: 0.0423
Epoch [19/20] Train Loss: 0.0405 Val Loss: 0.0422
Epoch [20/20] Train Loss: 0.0405 Val Loss: 0.0422
1

In [14]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune(server.index_set[i],ewc_flag=True,importance=0.02,fine_tune_epochs=20)
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
self.importance 0.02
Epoch [1/20] Train Loss: 0.0424 Val Loss: 0.0433
Epoch [2/20] Train Loss: 0.0422 Val Loss: 0.0431
Epoch [3/20] Train Loss: 0.0420 Val Loss: 0.0430
Epoch [4/20] Train Loss: 0.0420 Val Loss: 0.0429
Epoch [5/20] Train Loss: 0.0419 Val Loss: 0.0430
Epoch [6/20] Train Loss: 0.0419 Val Loss: 0.0429
Epoch [7/20] Train Loss: 0.0418 Val Loss: 0.0429
Epoch [8/20] Train Loss: 0.0418 Val Loss: 0.0429
Epoch [9/20] Train Loss: 0.0418 Val Loss: 0.0428
Epoch [10/20] Train Loss: 0.0418 Val Loss: 0.0428
Epoch [11/20] Train Loss: 0.0417 Val Loss: 0.0428
Epoch [12/20] Train Loss: 0.0417 Val Loss: 0.0427
Epoch [13/20] Train Loss: 0.0417 Val Loss: 0.0429
Epoch [14/20] Train Loss: 0.0417 Val Loss: 0.0426
Epoch [15/20] Train Loss: 0.0417 Val Loss: 0.0428
Epoch [16/20] Train Loss: 0.0416 Val Loss: 0.0427
Epoch [17/20] Train Loss: 0.0416 Val Loss: 0.0429
Epoch [18/20] Train Loss: 0.0416 Val Loss: 0.0429
Epoch [19/20] Train Loss: 0.0416 Val Loss: 0.0427
Epoch [20/20] Train Loss: 0.04

In [61]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss[server.index_set[i]])
    fed_local_preds.append(fed_local_pred[server.index_set[i]])
    fed_local_models.append(fed_local_model[server.index_set[i]])
print(fed_local_losses)

[0.07019277994380627, 0.06390050535205088, 0.07960541857636139, 0.07376481334946744, 0.07278922048384605, 0.06719664158299565, 0.08431435201623261]


In [86]:

class Winkler_score():
    def __init__(self, alpha: float, reduction: str = 'none'):
        self.reduction = reduction
        self.alpha = alpha

    def score(self,l_interval,u_interval,y_t):
        score = u_interval - l_interval
        if y_t < l_interval:
            score += ((2/self.alpha) * (l_interval - y_t))
        elif y_t > u_interval:
            score += ((2/self.alpha) * (y_t - u_interval))
        return score

    def __call__(self, output: torch.Tensor, target: torch.Tensor):
        if self.alpha==0.2:
            u_interval_list = output[:,-1]
            l_interval_list = output[:,0]
        elif self.alpha==0.4:
            u_interval_list = output[:,-2]
            l_interval_list = output[:,1]
        elif self.alpha==0.6:
            u_interval_list = output[:,-3]
            l_interval_list = output[:,2]
        elif self.alpha==0.8:
            u_interval_list = output[:,-4]
            l_interval_list = output[:,3]

        loss = torch.Tensor([
            self.score(l_interval, u_interval, y_t) 
            for l_interval, u_interval, y_t in zip(l_interval_list, u_interval_list, target)
        ])
        if self.reduction == 'sum':
            loss = loss.sum()
        if self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [89]:
actual_y_lst=[]

for i in range(args_train.number_clients):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths='wf'+str(i+1)
    test_data, test_loader = get_data(args_temp,flag='test')
    actual_y=[]
    for idx, (seq_x, seq_x_concat, seq_y) in enumerate(test_loader):
        actual_y.append(seq_y)
    actual_y = torch.cat([torch.flatten(t) for t in actual_y])
    actual_y_lst.append(actual_y)


In [90]:
fed_local_winkler_2=[]
fed_local_winkler_4=[]
fed_local_winkler_6=[]

for i in range(args_train.number_clients):
    fed_local_winkler_2.append(Winkler_score(0.2,'mean')(fed_local_preds[i],actual_y_lst[i]))
    fed_local_winkler_4.append(Winkler_score(0.4,'mean')(fed_local_preds[i],actual_y_lst[i]))
    fed_local_winkler_6.append(Winkler_score(0.6,'mean')(fed_local_preds[i],actual_y_lst[i]))

In [92]:
print(fed_local_winkler_2)
print(fed_local_winkler_4)
print(fed_local_winkler_6)

[tensor(0.8785), tensor(0.7435), tensor(0.9665), tensor(0.9021), tensor(0.8371), tensor(0.8214), tensor(1.0217)]
[tensor(0.6499), tensor(0.5831), tensor(0.7332), tensor(0.6827), tensor(0.6639), tensor(0.6228), tensor(0.7800)]
[tensor(0.5210), tensor(0.4808), tensor(0.5943), tensor(0.5492), tensor(0.5455), tensor(0.5000), tensor(0.6287)]


In [14]:
import pickle

# Save the server object
with open('../result/4/server.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/4/clients.pkl', 'wb') as f:
    pickle.dump(clients, f)